In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from stock_pull import*

C:\Users\andre\Documents\Bootcamp\Class_repository\repository_work\Project_2_dream_team\stock_pull.py:11: UserWarning: Make sure you are in the PyViz environment
  warnings.warn('Make sure you are in the PyViz environment')
C:\Users\andre\Documents\Bootcamp\Class_repository\repository_work\Project_2_dream_team\stock_pull.py:23: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')


In [4]:
cleaned.head(12)

,SPY Open,SPY Close,SPY Volume
date,,,
2000-07-03,145.4375,147.2812,1436600.0
2000-07-05,146.3750,144.6250,2748200.0
2000-07-06,144.9375,145.7500,5963200.0
2000-07-07,146.6875,148.0937,3034800.0
2000-07-10,147.8750,147.8437,2816100.0
2000-07-11,147.4687,148.1562,5431600.0
2000-07-12,149.2812,149.1250,5883000.0
2000-07-13,149.9843,149.7812,5356000.0
2000-07-14,150.4375,151.2500,5341900.0
